### Youtube Sentence Adjust Word List Ratio

In [1]:
import pandas as pd
import numpy as np
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import glob
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian, Intersect ==> native language

# parameter
word_num = 28
file_ext = "28_Ety_Prefix"
adjust_sent_word_ratio = 100
adjust_videoid_text_word_ratio = 50
sample_sent_num = 6
time_shift = 0.6

In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

#### Calculate Sentence Ratio By Word

In [5]:
# original part
#df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
#df_adjust_word = df_word.iloc[0:word_num,]

# variable part
#df_ety_prefix_intersect = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/\
#{lang_folder.capitalize()} Intersect/{lang_folder.capitalize()}_Intersect_Native_And_Shared_Word_Prefix_Suffix_Custom_Concat.xlsx")
#df_adjust_word = df_ety_prefix_intersect

df_ety_prefix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Native_And_Shared_Word_Prefix_Suffix_Custom_Concat.xlsx")
df_adjust_word = df_ety_prefix

df_adjust_word

,search_word,word
0,ama,ama
1,bana,bana
2,ben,ben
3,ben,bence
4,ben,bende
...,...,...
6355,şut,şutu
6356,şut,şutunu
6357,şırınga,şırınga
6358,şırınga,şırıngayla


In [6]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [7]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(df_adjust_word["word"].to_list())
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [8]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id",f"{file_ext}_word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,44.5,"özgür, kullanılıyor, büyük, araçlar, kamyon","bunlar, normalde, daha, için"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"burcu, arka, bağlamak, tarafı, olmayacak, kolay",
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,20.1,"yarım, arabaya, attıracağım, özgür",tur
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,22.3,"burcu, tamam, oldu, devam, et","daha, biraz"
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"tekere, burcu, iki, kar, şimdilik, artarsa, ta...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,33.4,"ki, herkes, umarız, ellerde, güçlü, büyük, yük...","ve, bu, teknoloji, için"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"halde, yararlanacağı, herkesin, kurumlarda, fa...","ve, bir, daha"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,16.8,"olarak, belirttim, yararlandığım, açıklamada, ...",link


In [9]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,44.5,"özgür, kullanılıyor, büyük, araçlar, kamyon","bunlar, normalde, daha, için"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"burcu, arka, bağlamak, tarafı, olmayacak, kolay",NaN
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,20.1,"yarım, arabaya, attıracağım, özgür",tur
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,22.3,"burcu, tamam, oldu, devam, et","daha, biraz"
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"tekere, burcu, iki, kar, şimdilik, artarsa, ta...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,33.4,"ki, herkes, umarız, ellerde, güçlü, büyük, yük...","ve, bu, teknoloji, için"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"halde, yararlanacağı, herkesin, kurumlarda, fa...","ve, bir, daha"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,16.8,"olarak, belirttim, yararlandığım, açıklamada, ...",link


In [10]:
df_sentence_ratio_result.to_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_All_Ratio_Result.csv", index=False)

#### Select Adjust Sentence Ratio Result

In [11]:
df_sentence_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_All_Ratio_Result.csv")
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,44.5,"özgür, kullanılıyor, büyük, araçlar, kamyon","bunlar, normalde, daha, için"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"burcu, arka, bağlamak, tarafı, olmayacak, kolay",NaN
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,20.1,"yarım, arabaya, attıracağım, özgür",tur
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,22.3,"burcu, tamam, oldu, devam, et","daha, biraz"
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"tekere, burcu, iki, kar, şimdilik, artarsa, ta...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,33.4,"ki, herkes, umarız, ellerde, güçlü, büyük, yük...","ve, bu, teknoloji, için"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"halde, yararlanacağı, herkesin, kurumlarda, fa...","ve, bir, daha"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,16.8,"olarak, belirttim, yararlandığım, açıklamada, ...",link


In [12]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result[f"{file_ext}_word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word
0,222,00:27:46.048,00:27:46.560,evet,8V9tq1pe8eI,100.1,NaN,evet
1,306,00:38:35.008,00:38:36.032,oralar da,8V9tq1pe8eI,100.1,NaN,"oralar, da"
2,488,00:03:23.020,00:03:24.200,krema gibi,cQjX9DzXTw0,100.1,NaN,"gibi, krema"
3,505,00:04:20.140,00:04:28.120,kremayı da,cQjX9DzXTw0,100.1,NaN,"da, kremayı"
4,510,00:04:48.400,00:04:49.280,o da,cQjX9DzXTw0,100.1,NaN,"da, o"
...,...,...,...,...,...,...,...,...
53245,3036709,00:17:25.970,00:17:26.000,bu,sSTIfQUm18Q,100.1,NaN,bu
53246,3036732,00:20:21.720,00:20:21.780,ben,sSTIfQUm18Q,100.1,NaN,ben
53247,3036760,00:25:30.370,00:25:30.400,bu,sSTIfQUm18Q,100.1,NaN,bu
53248,3036762,00:25:54.640,00:25:54.700,sen,sSTIfQUm18Q,100.1,NaN,sen


In [13]:
df_adjust_sentence_ratio.sort_values("sentence",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word
13825,826662,00:50:41.954,00:50:43.795,ne kadar basit bir şey değil mi bu senin için,OE8ujtE5ybI,100.1,NaN,"bu, basit, için, bir, ne, senin, kadar, şey, m..."
20466,1183737,01:37:58.288,01:38:01.598,finansal performansın objektif analizi için,awe0Thv_RQw,100.1,NaN,"analizi, performansın, için, objektif, finansal"
19074,1107468,00:17:56.080,00:17:57.780,senin için çok normal bir şey değil mi bu,kSIzlvs8ngw,100.1,NaN,"bu, için, senin, bir, şey, mi, normal, çok, değil"
41223,2225948,01:23:00.940,01:23:03.100,hayır senin gibi diplomam da yok benim,e7RXMoofC1A,100.1,NaN,"gibi, senin, diplomam, da, benim, hayır, yok"
25092,1410731,00:03:49.693,00:03:51.693,ne polyester gibi bir şey mi pantolon,gDpy9EhhyTk,100.1,NaN,"gibi, ne, bir, şey, polyester, mi, pantolon"
...,...,...,...,...,...,...,...,...
48123,2671464,00:19:24.610,00:19:30.340,o,RLYKwhXlF98,100.1,NaN,o
11116,685518,00:06:21.260,00:06:21.960,o,ANayzbybVDA,100.1,NaN,o
16714,974649,01:12:37.724,01:12:38.801,o,9Ez5806fqiw,100.1,NaN,o
52240,2999064,00:20:16.900,00:20:17.900,o,rCWONmYFOCc,100.1,NaN,o


In [14]:
#df_adjust_sentence_ratio.to_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
#{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv", index=False)

In [15]:
df_adjust_sentence_ratio.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.xlsx", index=False)

#### Calculate Videoid Text Ratio By Word

In [4]:
# original part
#df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
#df_adjust_word = df_word.iloc[0:word_num,]

# variable part
#df_ety_prefix_intersect = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/\
#{lang_folder.capitalize()} Intersect/{lang_folder.capitalize()}_Intersect_Native_And_Shared_Word_Prefix_Suffix_Custom_Concat.xlsx")
#df_adjust_word = df_ety_prefix_intersect

df_ety_prefix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Native_And_Shared_Word_Prefix_Suffix_Custom_Concat.xlsx")
df_adjust_word = df_ety_prefix

df_adjust_word

,search_word,word
0,ama,ama
1,bana,bana
2,ben,ben
3,ben,bence
4,ben,bende
...,...,...
6355,şut,şutu
6356,şut,şutunu
6357,şırınga,şırınga
6358,şırınga,şırıngayla


In [5]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [6]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

,index,video_id,sentence
0,0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...
1,1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...
2,2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...
3,3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...
4,4,00fr8yyBQYo,bill gates and george soros developed a new co...
...,...,...,...
17990,17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...
17991,17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...
17992,17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...
17993,17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...


In [7]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(df_adjust_word["word"].to_list())
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [8]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence",f"{file_ext}_word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

,video_id,sentence,28_Ety_Prefix_word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,8.4,"kaşıkla, yapılışını, arasında, kaşarı, boza, k...","bugün, gibi, var, da, bunu, de, o, çok, bu, ka..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,4.5,"başkanlarımızdan, muğla, çalışmaları, biliyor,...","kampanyası, var, şey, ne, mi, budur, sivil, tü..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,15.7,"iyap, teklifimdeni, iselvii, yoksan, l, seviyo...","ne, mi, ben, de, sen, benim, biraz"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,20.5,"tanıdım, ablam, kendinle, ya, olan, terk, hala...","var, ne, mi, beni, ama, senin, da, gibi, onu, ..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.8,"see, building, coming, writer, so, something, ...","pentagon, like, not, risk, reset, test"
...,...,...,...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,6.5,"iklimini, olmayi, çimde, sıcağı, yeni, beyne, ...","fazla, sistemine, var, vitamini, ne, kivi, ben..."
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,23.7,"diğer, yaklaşırsın, cinayetler, bırak, ya, ola...","varsa, gibi, da, mi, senden, yoksa, ben, de, s..."
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,3.9,"bahsedeceğiz, gördüğümüz, larımız, çıkamıyoruz...","mi, bence, elektrikli, ama, mayın, buranın, bu..."
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,25.7,"hikaye, geçiş, fracchia, taslağını, küçük, vid...","kontrol, bir, film, bunun, ben, senaryoyu, çok..."


In [9]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

,video_id,sentence,28_Ety_Prefix_word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,8.4,"kaşıkla, yapılışını, arasında, kaşarı, boza, k...","bugün, gibi, var, da, bunu, de, o, çok, bu, ka..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,4.5,"başkanlarımızdan, muğla, çalışmaları, biliyor,...","kampanyası, var, şey, ne, mi, budur, sivil, tü..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,15.7,"iyap, teklifimdeni, iselvii, yoksan, l, seviyo...","ne, mi, ben, de, sen, benim, biraz"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,20.5,"tanıdım, ablam, kendinle, ya, olan, terk, hala...","var, ne, mi, beni, ama, senin, da, gibi, onu, ..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.8,"see, building, coming, writer, so, something, ...","pentagon, like, not, risk, reset, test"
...,...,...,...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,6.5,"iklimini, olmayi, çimde, sıcağı, yeni, beyne, ...","fazla, sistemine, var, vitamini, ne, kivi, ben..."
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,23.7,"diğer, yaklaşırsın, cinayetler, bırak, ya, ola...","varsa, gibi, da, mi, senden, yoksa, ben, de, s..."
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,3.9,"bahsedeceğiz, gördüğümüz, larımız, çıkamıyoruz...","mi, bence, elektrikli, ama, mayın, buranın, bu..."
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,25.7,"hikaye, geçiş, fracchia, taslağını, küçük, vid...","kontrol, bir, film, bunun, ben, senaryoyu, çok..."


In [22]:
#df_text_ratio_result.to_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
#{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Videoid_Text_{file_ext}_Word_All_Ratio_Result.csv", index=False)

In [23]:
df_text_ratio_result.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Videoid_Text_{file_ext}_Word_All_Ratio_Result.xlsx", index=False)

In [ ]:
# 2 Way
#df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
#df_videoid_sentence

In [ ]:
#def videoid_word_ratio(df_word, df_videoid_sentence, word_column_name, videoid_sentence_column_name, word_num):
#    '''
#    videoid_word_ratio(df_adjust_word, df_videoid_sentence, "word", "sentence", word_num)
#    '''
#    word_set = set(df_word[f"{word_column_name}"].to_list())
#    for i in range(len(df_videoid_sentence)):
#        text = df_videoid_sentence.loc[i,f"{videoid_sentence_column_name}"]
#        text_word = re.findall(r"\w+", text, re.UNICODE)
#        text_word_set = set(text_word)
#        intersect_word = list(word_set.intersection(text_word_set))
#        different_word = list(text_word_set.difference(word_set))
#        df_videoid_sentence.loc[i,f"{word_num}_word_ratio"] = round(((len(intersect_word)/len(text_word)+0.001)*100),)
#        df_videoid_sentence.loc[i,"different_word"] = ", ".join(different_word)
#        df_videoid_sentence.loc[i,"intersect_word"] = ", ".join(intersect_word)
#        
#    return df_videoid_sentence

In [ ]:
#df_text_ratio_result = videoid_word_ratio(df_adjust_word, df_videoid_sentence, "word", "sentence", word_num)
#df_text_ratio_result

In [ ]:
#df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
#df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
#df_text_ratio_result 

In [ ]:
#df_text_ratio_result.to_csv(f"Youtube_Videoid_Text_{file_ext}_Word_All_Ratio_Result.csv", index=False)

#### Select Adjust Videoid Text Ratio Result

In [10]:
df_text_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Videoid_Text_{file_ext}_Word_All_Ratio_Result.csv")
df_text_ratio_result

,video_id,sentence,28_Ety_Prefix_word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,8.4,"saat, olarak, arkadaşlar, göstermek, sizlerde,...","bu, o, ve, gibi, için, bunu, bugün, ama, var, ..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,4.5,"aradığımız, düşünmüşler, konuşmak, kuruluşuyla...","sivil, türkiye, burada, medya, ve, bunları, ba..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,15.7,"isen, yaşamaktansa, idünyalara, ihem, şu, gere...","biraz, ne, ben, mi, sen, benim, de"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,20.5,"evine, kendinle, hala, usta, bilmiyorum, yakın...","mısın, çok, gibi, ne, bir, onu, var, sen, bu, ..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.8,"appropriate, unable, addressed, dealing, us, s...","pentagon, test, reset, risk, like, not"
...,...,...,...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,6.5,"gala, ederimah, oldukça, rize, suyunu, sever, ...","avrupa, buradan, biraz, ve, bana, antioksidan,..."
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,23.7,"bilmem, hanım, hanımı, en, anlatmaya, alamaz, ...","bu, varsa, o, gibi, bana, senden, ben, birisi,..."
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,3.9,"balkonumuz, lansmandan, kapıyı, göstermek, değ...","onun, site, bunun, onu, bugün, burayı, marka, ..."
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,25.7,"yaptığıma, taslağını, yelpazede, reklamlardan,...","senaryoyu, için, ve, bir, ben, bunun, kontrol,..."


In [11]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result[f"{file_ext}_word_ratio"] >= adjust_videoid_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

,video_id,sentence,28_Ety_Prefix_word_ratio,different_word,intersect_word
0,0gPC8dJJqW8,müzik,100.1,NaN,müzik
1,177APXoEeXo,şarkı ve müzik,66.8,şarkı,"ve, müzik"
2,2KQaODuI838,basit poğaça,50.1,poğaça,basit
3,2LqpcYH8BVk,bu,100.1,NaN,bu
4,2WghnknNIMQ,diy minyatür karton ev,50.1,"ev, diy","minyatür, karton"
...,...,...,...,...,...
108,x6UBFpuVFcQ,diy minyatür karton ev,50.1,"ev, diy","minyatür, karton"
109,x97FOvJ_n_E,detaylı ve altyazılı tarifi için bir önceki vi...,55.7,"tarifi, altyazılı, izleyin, önceki","detaylı, ve, için, bir, videoyu"
110,y0yqDeLBq0U,aslan burcu kahve fali,50.1,"fali, burcu","aslan, kahve"
111,z9zuXrnIixk,diy minyatür karton ev,50.1,"ev, diy","minyatür, karton"


In [12]:
df_adjust_text_ratio.sort_values("sentence",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_adjust_text_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,video_id,sentence,28_Ety_Prefix_word_ratio,different_word,intersect_word
22,DGXeHHqjpp0,sorarlarsa seni bana bir gülüşü vardı onu da b...,53.4,"eski, sorarlarsa, sözleri, dersin, sevgiliye, ...","sitem, bana, bir, ben, onu, da, seni, vardı"
58,WJY9mOwEFSE,karşınızda adam mı var hemen uygun bir pozisyo...,58.4,"adam, uygun, vurun, karşınızda, hemen","bu, bir, pozisyonda, onu, var, kadar, mı"
28,FcexaFOs4FE,daha çok kişiye ulaşabilmek için videoyu beğen...,50.1,"seviniriz, kişiye, beğenirseniz, ulaşabilmek","çok, videoyu, daha, için"
87,mjkC889KoQM,tereyağı fıstık ezmesi pudra şekeri bisküvi çi...,50.1,"sıvıyağ, tereyağı, ezmesi, fıstık","bisküvi, şekeri, pudra, çikolata"
109,x97FOvJ_n_E,detaylı ve altyazılı tarifi için bir önceki vi...,55.7,"tarifi, altyazılı, izleyin, önceki","detaylı, ve, için, bir, videoyu"
...,...,...,...,...,...
52,TosArq0B6S0,müzik,100.1,NaN,müzik
43,PnwSr5f9w1U,müzik,100.1,NaN,müzik
45,RE5IgCDyujE,müzik,100.1,NaN,müzik
0,0gPC8dJJqW8,müzik,100.1,NaN,müzik


In [27]:
#df_adjust_text_ratio.to_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
#{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Videoid_{file_ext}_Word_{adjust_videoid_text_word_ratio}_Ratio_Text.csv", index=False)

In [13]:
df_adjust_text_ratio.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Videoid_{file_ext}_Word_{adjust_videoid_text_word_ratio}_Ratio_Text.xlsx", index=False)

In [14]:
df_text_ratio_result.sort_values(by=f"{file_ext}_word_ratio", ascending=False)

,video_id,sentence,28_Ety_Prefix_word_ratio,different_word,intersect_word
15508,rFTWpjtFNQ8,android,100.1,NaN,android
5698,Jv5n18MD32M,çok basit,100.1,NaN,"çok, basit"
10514,_fwSPHnO4eM,müzik,100.1,NaN,müzik
1426,548RqUIdzkg,finale,100.1,NaN,finale
14953,pM74dDjMt4s,müzik,100.1,NaN,müzik
...,...,...,...,...,...
16126,tPxoODqd9BE,pergola tente altına uyguladığımız albert gena...,0.1,"cambalkon, tente, pergola, albert, isıcamlı, a...",NaN
10952,b8aN6gakrbY,abone olur musun,0.1,"olur, musun, abone",NaN
6130,LQ9xsGygma8,fırında afiyet olsun,0.1,"afiyet, olsun, fırında",NaN
1758,6Kni_bfwDjM,aeoni um çeşi tleri kendi çoğaltip büyüttüğüm ...,0.1,"um, çoğaltip, çeşi, bakimi, umlar, büyüttüğüm,...",NaN


#### Calculate Sequential Sentence Length Of Adjust Ratio Result

In [30]:
df_sentence_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_All_Ratio_Result.csv")
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,44.5,"özgür, kullanılıyor, büyük, araçlar, kamyon","bunlar, normalde, daha, için"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"burcu, arka, bağlamak, tarafı, olmayacak, kolay",NaN
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,20.1,"yarım, arabaya, attıracağım, özgür",tur
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,22.3,"burcu, tamam, oldu, devam, et","daha, biraz"
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"tekere, burcu, iki, kar, şimdilik, artarsa, ta...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,33.4,"ki, herkes, umarız, ellerde, güçlü, büyük, yük...","ve, bu, teknoloji, için"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"halde, yararlanacağı, herkesin, kurumlarda, fa...","ve, bir, daha"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,16.8,"olarak, belirttim, yararlandığım, açıklamada, ...",link


In [31]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].astype("object")
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].astype("object")

In [32]:
df_adjust_ratio = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio

,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word
0,826662,00:50:41.954,00:50:43.795,ne kadar basit bir şey değil mi bu senin için,OE8ujtE5ybI,100.1,NaN,"bu, basit, için, bir, ne, senin, kadar, şey, m..."
1,1183737,01:37:58.288,01:38:01.598,finansal performansın objektif analizi için,awe0Thv_RQw,100.1,NaN,"analizi, performansın, için, objektif, finansal"
2,1107468,00:17:56.080,00:17:57.780,senin için çok normal bir şey değil mi bu,kSIzlvs8ngw,100.1,NaN,"bu, için, senin, bir, şey, mi, normal, çok, değil"
3,2225948,01:23:00.940,01:23:03.100,hayır senin gibi diplomam da yok benim,e7RXMoofC1A,100.1,NaN,"gibi, senin, diplomam, da, benim, hayır, yok"
4,1410731,00:03:49.693,00:03:51.693,ne polyester gibi bir şey mi pantolon,gDpy9EhhyTk,100.1,NaN,"gibi, ne, bir, şey, polyester, mi, pantolon"
...,...,...,...,...,...,...,...,...
53245,2671464,00:19:24.610,00:19:30.340,o,RLYKwhXlF98,100.1,NaN,o
53246,685518,00:06:21.260,00:06:21.960,o,ANayzbybVDA,100.1,NaN,o
53247,974649,01:12:37.724,01:12:38.801,o,9Ez5806fqiw,100.1,NaN,o
53248,2999064,00:20:16.900,00:20:17.900,o,rCWONmYFOCc,100.1,NaN,o


In [33]:
df_adjust_ratio["different_word"] = df_adjust_ratio["different_word"].astype("object")
df_adjust_ratio["intersect_word"] = df_adjust_ratio["intersect_word"].astype("object")

In [34]:
df_adjust_ratio["with_word"] = 1
df_adjust_ratio

,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word,with_word
0,826662,00:50:41.954,00:50:43.795,ne kadar basit bir şey değil mi bu senin için,OE8ujtE5ybI,100.1,NaN,"bu, basit, için, bir, ne, senin, kadar, şey, m...",1
1,1183737,01:37:58.288,01:38:01.598,finansal performansın objektif analizi için,awe0Thv_RQw,100.1,NaN,"analizi, performansın, için, objektif, finansal",1
2,1107468,00:17:56.080,00:17:57.780,senin için çok normal bir şey değil mi bu,kSIzlvs8ngw,100.1,NaN,"bu, için, senin, bir, şey, mi, normal, çok, değil",1
3,2225948,01:23:00.940,01:23:03.100,hayır senin gibi diplomam da yok benim,e7RXMoofC1A,100.1,NaN,"gibi, senin, diplomam, da, benim, hayır, yok",1
4,1410731,00:03:49.693,00:03:51.693,ne polyester gibi bir şey mi pantolon,gDpy9EhhyTk,100.1,NaN,"gibi, ne, bir, şey, polyester, mi, pantolon",1
...,...,...,...,...,...,...,...,...,...
53245,2671464,00:19:24.610,00:19:30.340,o,RLYKwhXlF98,100.1,NaN,o,1
53246,685518,00:06:21.260,00:06:21.960,o,ANayzbybVDA,100.1,NaN,o,1
53247,974649,01:12:37.724,01:12:38.801,o,9Ez5806fqiw,100.1,NaN,o,1
53248,2999064,00:20:16.900,00:20:17.900,o,rCWONmYFOCc,100.1,NaN,o,1


In [35]:
df_adjust_ratio_flag = pd.merge(df_sentence_ratio_result , df_adjust_ratio, how="left", on=["index","sentence","start_time","end_time","video_id",f"{file_ext}_word_ratio","different_word","intersect_word"])
df_adjust_ratio_flag["with_word"].fillna(0, inplace=True)
df_adjust_ratio_flag

,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word,with_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,44.5,"özgür, kullanılıyor, büyük, araçlar, kamyon","bunlar, normalde, daha, için",0.0
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"burcu, arka, bağlamak, tarafı, olmayacak, kolay",NaN,0.0
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,20.1,"yarım, arabaya, attıracağım, özgür",tur,0.0
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,22.3,"burcu, tamam, oldu, devam, et","daha, biraz",0.0
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"tekere, burcu, iki, kar, şimdilik, artarsa, ta...",ama,0.0
...,...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,33.4,"ki, herkes, umarız, ellerde, güçlü, büyük, yük...","ve, bu, teknoloji, için",0.0
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"halde, yararlanacağı, herkesin, kurumlarda, fa...","ve, bir, daha",0.0
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"i, teşekkürler, zlediğiniz",için,0.0
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,16.8,"olarak, belirttim, yararlandığım, açıklamada, ...",link,0.0


In [36]:
df_adjust_ratio_flag.with_word.value_counts()

0.0    2983687
1.0      53250
Name: with_word, dtype: int64

In [37]:
# Code Running For Video_id Sentence
zeros = []
ones = []
zero_list = []
one_list = []
for i in range(len(df_adjust_ratio_flag)):
    videoid = df_adjust_ratio_flag.loc[i,"video_id"]    
    if df_adjust_ratio_flag.loc[i,"with_word"] == 0:
        zeros.append(i)
        try:
            one_list.append((ones[0],len(ones)))
        except:
            pass
        ones = []
    else:
        ones.append(i)
        try:
            zero_list.append((zeros[0],len(zeros)))
        except:
            pass
        zeros = []

    try:
        if df_adjust_ratio_flag.loc[i+1,"video_id"] != videoid:
            try:
                one_list.append((ones[0],len(ones)))
                ones = []
            except:
                pass
            try:
                zero_list.append((zeros[0],len(zeros)))
                zeros = []
            except:
                pass
        else:
            pass
    except:
        pass

try:
    one_list.append((ones[0],len(ones)))
except:
    pass
try:
    zero_list.append((zeros[0],len(zeros)))
except:
    pass

In [38]:
df_one = pd.DataFrame(one_list)
df_one.rename(columns={0:"index",1:"length"}, inplace=True)
df_one.drop_duplicates(inplace=True)
df_one.sort_values(by="length", ascending=False, inplace=True)
df_one.reset_index(drop=True, inplace=True)
df_one

,index,length
0,2251858,7
1,1972884,7
2,1194079,6
3,1447811,6
4,1008806,6
...,...,...
50395,1053966,1
50396,1053984,1
50397,1054020,1
50398,1054031,1


In [39]:
df_one_all = pd.DataFrame()
for i in range(len(df_one)):
    index = df_one.loc[i,"index"]
    length = df_one.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_all = pd.concat([df_one_all, df_video],axis=0)
df_one_all.reset_index(inplace=True, drop=True)
df_one_all

,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word,with_word
0,2251858,00:40:46.760,00:40:47.420,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
1,2251859,00:40:47.580,00:40:48.700,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
2,2251860,00:40:49.240,00:40:50.380,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
3,2251861,00:40:51.000,00:40:52.040,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
4,2251862,00:40:52.740,00:40:53.840,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
...,...,...,...,...,...,...,...,...,...
53245,1053966,00:58:24.120,00:58:25.120,yok,e3rIly3dY5E,100.1,NaN,yok,1.0
53246,1053984,00:59:58.240,00:59:59.240,ama,e3rIly3dY5E,100.1,NaN,ama,1.0
53247,1054020,01:02:11.180,01:02:12.180,evet,e3rIly3dY5E,100.1,NaN,evet,1.0
53248,1054031,01:02:41.600,01:02:42.340,yok,e3rIly3dY5E,100.1,NaN,yok,1.0


In [ ]:
#df_one_all.to_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
#{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_{adjust_sent_word_ratio}_Ratio_Sequential_Sentences_And_Others.csv", index=False)

In [40]:
# Only Sequential Sentences
df_one_sequential = df_one[df_one.length > 1]
df_one_sequential

,index,length
0,2251858,7
1,1972884,7
2,1194079,6
3,1447811,6
4,1008806,6
...,...,...
2482,760373,2
2483,105724,2
2484,1552217,2
2485,76411,2


In [41]:
df_one_sequential_all = pd.DataFrame()
for i in range(len(df_one_sequential)):
    index = df_one_sequential.loc[i,"index"]
    length = df_one_sequential.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_sequential_all = pd.concat([df_one_sequential_all, df_video],axis=0)
df_one_sequential_all.reset_index(inplace=True, drop=True)
df_one_sequential_all

,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word,with_word
0,2251858,00:40:46.760,00:40:47.420,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
1,2251859,00:40:47.580,00:40:48.700,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
2,2251860,00:40:49.240,00:40:50.380,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
3,2251861,00:40:51.000,00:40:52.040,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
4,2251862,00:40:52.740,00:40:53.840,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
...,...,...,...,...,...,...,...,...,...
5332,1552218,00:16:32.224,00:16:32.999,ama,JaOgnPCTw_M,100.1,NaN,ama,1.0
5333,76411,00:07:48.091,00:07:53.134,ağustos,Ao3UnRqDeVg,100.1,NaN,ağustos,1.0
5334,76412,00:07:53.632,00:07:57.734,ama,Ao3UnRqDeVg,100.1,NaN,ama,1.0
5335,2171994,00:01:10.475,00:01:11.475,sensin,hQJp_QZ8Emg,100.1,NaN,sensin,1.0


In [42]:
#df_one_sequential_all.to_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
#{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_{adjust_sent_word_ratio}_Ratio_Only_Sequential_Sentences.csv", index=False)

In [43]:
#df_one_sequential_all.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
#{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_{adjust_sent_word_ratio}_Ratio_Only_Sequential_Sentences.xlsx", index=False)

#### Count Sentence Words By Adjust Column

In [44]:
df_one_sequential_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_{adjust_sent_word_ratio}_Ratio_Only_Sequential_Sentences.csv")
df_one_sequential_all

,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word,with_word
0,2251858,00:40:46.760,00:40:47.420,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
1,2251859,00:40:47.580,00:40:48.700,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
2,2251860,00:40:49.240,00:40:50.380,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
3,2251861,00:40:51.000,00:40:52.040,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
4,2251862,00:40:52.740,00:40:53.840,türkiye,Qhhka9zIUzI,100.1,NaN,türkiye,1.0
...,...,...,...,...,...,...,...,...,...
5332,1552218,00:16:32.224,00:16:32.999,ama,JaOgnPCTw_M,100.1,NaN,ama,1.0
5333,76411,00:07:48.091,00:07:53.134,ağustos,Ao3UnRqDeVg,100.1,NaN,ağustos,1.0
5334,76412,00:07:53.632,00:07:57.734,ama,Ao3UnRqDeVg,100.1,NaN,ama,1.0
5335,2171994,00:01:10.475,00:01:11.475,sensin,hQJp_QZ8Emg,100.1,NaN,sensin,1.0


In [45]:
def text_to_list_df(df_target, target_column): 
    '''
    df_target include dict value columns, target_column occurs dict \n
    text_to_list_df(df_one_sequential_all, "intersect_word")
    '''
    text_value_list = []
    for i in df_target[f"{target_column}"]:
        try:
            var = re.findall(r"\w+", i)
            for j in var:
                text_value_list.append(j)
        except:
            pass
    df_alone_value = pd.DataFrame(text_value_list)
    df_alone_value.rename(columns={0:f"{target_column}_alone"}, inplace=True)
    df_unique_count = pd.DataFrame(df_alone_value.value_counts(ascending=False))
    df_unique_count.reset_index(inplace=True)
    df_unique_count.rename(columns={0:"count"}, inplace=True)
            
    return df_unique_count

In [46]:
df_unique_word_count = text_to_list_df(df_one_sequential_all, "intersect_word")
df_unique_word_count

,intersect_word_alone,count
0,evet,702
1,ne,444
2,ben,370
3,alo,333
4,bu,325
...,...,...
495,like,1
496,liderin,1
497,köri,1
498,kuzenin,1


In [47]:
#df_unique_word_count.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
#{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_{adjust_sent_word_ratio}_Ratio_Sequential_Sentence_Words_Count.xlsx", index=False)

#### Count Adjust Sentence Ratio Result By Videoid

In [48]:
df_adjust_ratio = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio

,index,start_time,end_time,sentence,video_id,28_Ety_Prefix_word_ratio,different_word,intersect_word
0,826662,00:50:41.954,00:50:43.795,ne kadar basit bir şey değil mi bu senin için,OE8ujtE5ybI,100.1,NaN,"bu, basit, için, bir, ne, senin, kadar, şey, m..."
1,1183737,01:37:58.288,01:38:01.598,finansal performansın objektif analizi için,awe0Thv_RQw,100.1,NaN,"analizi, performansın, için, objektif, finansal"
2,1107468,00:17:56.080,00:17:57.780,senin için çok normal bir şey değil mi bu,kSIzlvs8ngw,100.1,NaN,"bu, için, senin, bir, şey, mi, normal, çok, değil"
3,2225948,01:23:00.940,01:23:03.100,hayır senin gibi diplomam da yok benim,e7RXMoofC1A,100.1,NaN,"gibi, senin, diplomam, da, benim, hayır, yok"
4,1410731,00:03:49.693,00:03:51.693,ne polyester gibi bir şey mi pantolon,gDpy9EhhyTk,100.1,NaN,"gibi, ne, bir, şey, polyester, mi, pantolon"
...,...,...,...,...,...,...,...,...
53245,2671464,00:19:24.610,00:19:30.340,o,RLYKwhXlF98,100.1,NaN,o
53246,685518,00:06:21.260,00:06:21.960,o,ANayzbybVDA,100.1,NaN,o
53247,974649,01:12:37.724,01:12:38.801,o,9Ez5806fqiw,100.1,NaN,o
53248,2999064,00:20:16.900,00:20:17.900,o,rCWONmYFOCc,100.1,NaN,o


In [49]:
df_adjust_ratio["different_word"] = df_adjust_ratio["different_word"].astype("object")
df_adjust_ratio["intersect_word"] = df_adjust_ratio["intersect_word"].astype("object")

In [50]:
df_ratio_count = pd.DataFrame(df_adjust_ratio.groupby("video_id")[f"{word_num}_word_ratio"].count())
df_ratio_count.reset_index(inplace=True)
df_ratio_count.rename(columns={f"{word_num}_word_ratio":f"{word_num}_word_ratio_count"}, inplace=True)
df_ratio_count.sort_values(by=f"{word_num}_word_ratio_count", ascending=False, inplace=True)
df_ratio_count.reset_index(drop=True,inplace=True)
df_ratio_count

,video_id,28_Ety_Prefix_word_ratio_count
0,eHtgG748qhI,196
1,qQUZ5d5mpWk,152
2,vLkbW6uukDU,145
3,WcRHKUhrmHQ,143
4,ofzGf0PwldY,129
...,...,...
6391,dCpYD0wWhew,1
6392,FJWNK3zlhAI,1
6393,dD3K4yeyEBI,1
6394,dDn3fCb_KAM,1


In [51]:
#df_ratio_count.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
#{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_{adjust_sent_word_ratio}_Ratio_Videoid_Count.xlsx", index=False)

#### Find Word In Sentence Time Interval

In [68]:
# original part
#df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
#df_adjust_word = df_word.iloc[0:word_num,]

# variable part
#df_ety_prefix_intersect = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/\
#{lang_folder.capitalize()} Intersect/{lang_folder.capitalize()}_Intersect_Native_And_Shared_Word_Prefix_Suffix_Custom_Concat.xlsx")
#df_adjust_word = df_ety_prefix_intersect

df_ety_prefix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Native_And_Shared_Word_Prefix_Suffix_Custom_Concat.xlsx")
df_adjust_word = df_ety_prefix

df_adjust_word

,search_word,word
0,ama,ama
1,bana,bana
2,ben,ben
3,ben,bence
4,ben,bende
...,...,...
6355,şut,şutu
6356,şut,şutunu
6357,şırınga,şırınga
6358,şırınga,şırıngayla


In [69]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [70]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [71]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3036932,471.970,472.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,472.470,482.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,482.498,484.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,484.178,488.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [72]:
word_list = df_adjust_word.iloc[:,0].values.tolist()

In [73]:
df_word_result = pd.DataFrame()
for i in word_list:
    try:
        word_in_video = df_youtube_sentence[df_youtube_sentence.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].sample(sample_sent_num)
    except:
        word_in_video = df_youtube_sentence[df_youtube_sentence.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(sample_sent_num) 
    word_in_video.insert(0,"word",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result = df_word_result.sort_values(by=["video_id","start_time"], ascending=True)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result

In [ ]:
word_time_loc_list = []
def word_time_loc(df):
    for i in range(len(df)):
        word = df.loc[i,"word"]
        start_time = df.loc[i,"start_time"]
        end_time = df.loc[i,"end_time"]
        sentence = df.loc[i,"sentence"]
        video_id = df.loc[i,"video_id"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=["word","start_time","end_time","sentence","video_id"])

    return df_word_time_loc

In [ ]:
df_word_time_loc_result = word_time_loc(df_word_result)
df_word_time_loc_result

,word,start_time,end_time,sentence,video_id
0,beni,1250.669250,1250.978150,hem var mı öyle beni sevgilimin karşısında küç...,0AOYUEuSiyw
1,sen,88.051136,88.374864,mehmet verince mesele oluyo sen verince mı mes...,0MgW9k9sUqA
2,de,2407.780000,2408.271429,sen sen de şey,0bXZlTcsWrQ
3,değil,754.136206,754.560000,burada oynadığımız tabu sadece yine bizler içi...,19dSMaFc1sE
4,yok,574.569000,574.862722,yok evet özellikle vergi dairesinin son zamanl...,1boYNWzDEVQ
...,...,...,...,...,...
163,mı,2178.146909,2178.353455,bak bir kere soracağım insan gibi cevap ver va...,xHEVAFJPdto
164,gibi,108.294034,108.830517,kalbinden geldiği gibi haykır,yxtHIufbkEQ
165,mi,1990.148000,1990.356000,bu da fare rolünü oynar değil mi bücürük,zJMggnPkep8
166,bu,304.520000,304.803393,bu çaldığımız ilmekler içten bu değil ilmekler...,zSDFSgPiY8Y


In [ ]:
df_word_time_loc_result.start_time = df_word_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_time_loc_result.end_time = df_word_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_time_loc_result

,word,start_time,end_time,sentence,video_id
0,beni,1250.069250,1251.578150,hem var mı öyle beni sevgilimin karşısında küç...,0AOYUEuSiyw
1,sen,87.451136,88.974864,mehmet verince mesele oluyo sen verince mı mes...,0MgW9k9sUqA
2,de,2407.180000,2408.871429,sen sen de şey,0bXZlTcsWrQ
3,değil,753.536206,755.160000,burada oynadığımız tabu sadece yine bizler içi...,19dSMaFc1sE
4,yok,573.969000,575.462722,yok evet özellikle vergi dairesinin son zamanl...,1boYNWzDEVQ
...,...,...,...,...,...
163,mı,2177.546909,2178.953455,bak bir kere soracağım insan gibi cevap ver va...,xHEVAFJPdto
164,gibi,107.694034,109.430517,kalbinden geldiği gibi haykır,yxtHIufbkEQ
165,mi,1989.548000,1990.956000,bu da fare rolünü oynar değil mi bücürük,zJMggnPkep8
166,bu,303.920000,305.403393,bu çaldığımız ilmekler içten bu değil ilmekler...,zSDFSgPiY8Y


In [ ]:
df_word_time_loc_result.start_time = df_word_time_loc_result.start_time.apply(lambda x: round(x))
df_word_time_loc_result.end_time = df_word_time_loc_result.end_time.apply(lambda x: round(x))
df_word_time_loc_result 

,word,start_time,end_time,sentence,video_id
0,beni,1250,1252,hem var mı öyle beni sevgilimin karşısında küç...,0AOYUEuSiyw
1,sen,87,89,mehmet verince mesele oluyo sen verince mı mes...,0MgW9k9sUqA
2,de,2407,2409,sen sen de şey,0bXZlTcsWrQ
3,değil,754,755,burada oynadığımız tabu sadece yine bizler içi...,19dSMaFc1sE
4,yok,574,575,yok evet özellikle vergi dairesinin son zamanl...,1boYNWzDEVQ
...,...,...,...,...,...
163,mı,2178,2179,bak bir kere soracağım insan gibi cevap ver va...,xHEVAFJPdto
164,gibi,108,109,kalbinden geldiği gibi haykır,yxtHIufbkEQ
165,mi,1990,1991,bu da fare rolünü oynar değil mi bücürük,zJMggnPkep8
166,bu,304,305,bu çaldığımız ilmekler içten bu değil ilmekler...,zSDFSgPiY8Y


In [ ]:
df_word_time_loc_result.to_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/6-Youtube Sentence/\
{lang_folder.capitalize()} {lang_pair.capitalize()}/Youtube_Sentence_{file_ext}_Word_Time_Interval.xlsx", index=False)